# Image Classification Test Model

In [ ]:
#---- Install stuff -------
#!pip install --upgrade pip

#!pip install opencv-python
#!pip install tensorflow
#!pip install numpy==1.19.5

#!pip install Pillow
#!pip install playsound
#!pip install gTTS
#!pip install pyobjc

#!pip install matplotlib
#!pip install pandas
#!pip install seaborn

#!pip install PyYAML

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import time
from datetime import datetime
import random
from glob import glob
import os

In [ ]:
import cv2
from PIL import Image

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model

In [ ]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#print system info
import sys
print('Python: ',sys.version)

In [ ]:
#print tensor and keras version
print('keras: ', keras.__version__)
print('tensorflow: ', tf.__version__)
import numpy
print('numpy: ', numpy.version.version)

## Functions

In [ ]:
import FileIOUtil
import TTSUtil
from ImageClassificationCfg import ImageClassificationCfg
from LoadAndTestCNNUtil import LoadAndTestCNNUtil

# Read Cfg

In [ ]:
#cfg_file = './project/flowers-recognition.yml'
#cfg_file = './project/work_pose.yml'
cfg_file = './project/home_presence.yml'

cfg_file = ImageClassificationCfg(cfg_file)
cfg_file.load()

In [ ]:
cfg_file.log_info()

In [ ]:
def get_capture_dim(src_video):
    cap = None
    try:
        cap = cv2.VideoCapture(src_video)
        if(cap.isOpened()):
            ret, frame = cap.read()
            return frame.shape
        else:
            return 0,0
    finally:
        if cap!=None:
            cap.release()
        cv2.destroyAllWindows()

In [ ]:
#org shape 1080, 1920
capture_dim = get_capture_dim(cfg_file.cfg['src_video'])
print('org dim: ',capture_dim)

img_h = capture_dim[0]
img_w = capture_dim[1]

img_h = img_h//cfg_file.cfg['reduce_image_wh_by']
img_w = img_w//cfg_file.cfg['reduce_image_wh_by']

print('new dim h:',img_h)
print('new dim w:',img_w)

In [ ]:
def plot_image(images, captions=None, cmap=None):    
    if captions!=None:
        print(captions)
    
    if len(images) > 1:
        f, axes = plt.subplots(1, len(images), sharey=True, figsize=(4,4))
        f.set_figwidth(15)
        for ax,image in zip(axes, images):
            ax.imshow(image, cmap)
    else:
        plt.figure(figsize=(4,4))
        plt.imshow(images[0])
    plt.show()

In [ ]:
def get_file_name():
    return 'img_'+datetime.now().strftime("%Y%m%d%H%M%S")
#get_file_name()

In [ ]:
def capture_frames(cfg_file, crop=True, count=1, delay_sec=60, save=True):
    cap = None
    if img_h==0 or img_w==0:
        raise Exception('img_h or img_w can not be 0')
    try:
        cap = cv2.VideoCapture(cfg_file.cfg['src_video'])
        #cap.set(cv2.CAP_PROP_BUFFERSIZE, 1)
        
        if(cap.isOpened()):
            for i in range(0, count):
                ret, frame = cap.read()
                if not ret:
                    print("failed to grab frame ")
                    #raise Exception("failed to grab frame ")
                    return None, None

                #print('org shape: ',frame.shape)
                frame = cv2.resize(frame, (img_w,img_h), 
                                   interpolation = cv2.INTER_AREA)

                if crop:
                    if cfg_file.cfg['crop_image_from_left']>0:
                        new_wl = int(img_w*cfg_file.cfg['crop_image_from_left'])
                    else:
                        new_wl = 0
                        
                    if cfg_file.cfg['crop_image_from_right']>0:
                        #crop 70% on width from right
                        new_wr = int(img_w*cfg_file.cfg['crop_image_from_right'])
                    else:
                        new_wr = img_w                    
                        
                    frame = frame[0:img_h,new_wl:new_wr]

                if save:
                    img_name = cfg_file.loc_unknown+"frame_{0}.png".format(get_file_name())
                    cv2.imwrite(img_name, frame)
                else:
                    img_name = None

                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                
                if count>1 and save:
                    print('file saved: ',img_name)
                    
                if (i+1)!=count:
                    #close and re-open else we will get old frame
                    if cap != None:
                        cap.release()
                    #it take about 2 sec to open cam again
                    if delay_sec>0:
                        time.sleep(delay_sec-2) 
                    cap = cv2.VideoCapture(cfg_file.cfg['src_video'])
                    if(cap.isOpened()==False):
                        print('Could not open camera!')
                        break            
            return frame, img_name
        else:
            print('Could not open camera!')
            return None, None
    finally:
        if cap != None:
            cap.release()
        cv2.destroyAllWindows()

# Load Model

In [ ]:
cnn = LoadAndTestCNNUtil(cfg_file)
cnn.debug = True
cnn.load()

# Load Test Image

In [ ]:
print('project_dir: ',cfg_file.project_dir)

In [ ]:
all_images = glob(cfg_file.project_dir+'*/'+cfg_file.cfg['file_ext'])
print('all img count: ',len(all_images))

In [ ]:
frame = None
file = None

#file = random.choice(all_images)
frame, file = capture_frames(cfg_file=cfg_file,crop=True, count=1, delay_sec=0, save=True)
#file = './project/home_presence/no-one/frame_img_20210713121302.png'

print('file: ',file)
if frame is None:
    frame = Image.open(file)

plot_image([frame])

(img_class, model_acc) = cnn.predict(file)

In [ ]:
#delete temp file
if file.startswith(cfg_file.loc_unknown):
    print('delte temp file')
    os.remove(file)